## Этапы:
- [Скачиваю файл](#cell1)
- [Разбиваю данные на выборки](#cell2)
- [Исследую разные модели](#cell3)
- [Проверяю модель на тестовой выборке](#cell4)
- [Проверяю модель на адекватность](#cell5)

# 1. Скачиваю файл 
<a id="cell1"><a/>

In [3]:
import pandas as pd
data = pd.read_csv('/datasets/users_behavior.csv')
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


# 2. Разбиваю данные на выборки <a id="cell2"><a/>

In [4]:
from sklearn.model_selection import train_test_split
data_train, data_buf = train_test_split(data, test_size=0.40, random_state=12345)
data_test, data_valid = train_test_split(data_buf, test_size=0.50, random_state=12345)
print(data_train.shape, data_test.shape, data_valid.shape)
features_train, features_valid, features_test = data_train.drop('is_ultra', axis=1), data_valid.drop('is_ultra', axis=1), data_test.drop('is_ultra', axis=1)
target_train, target_valid, target_test = data_train['is_ultra'], data_valid['is_ultra'], data_test['is_ultra'] 


(1928, 5) (643, 5) (643, 5)


# 3. Исследую модели <a id="cell3"><a/>

Пробую дерево решений

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
best_model = [0, 0]
for depth in range (2, 12):
    model = DecisionTreeClassifier(random_state=12345, max_depth = depth)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    acc = accuracy_score(target_valid, predicted_valid)
    if acc > best_model[0]:
        best_model[0] = acc
        best_model[1] = model
best_model

[0.7993779160186625,
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False,
                        random_state=12345, splitter='best')]

Пробую случайный лес

In [6]:
from sklearn.ensemble import RandomForestClassifier
for estim in range(10, 51, 10):
    for depth in range (2, 12):
        model=RandomForestClassifier(random_state=12345, n_estimators=estim, max_depth=depth)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        acc = accuracy_score(target_valid, predicted_valid)
        if acc > best_model[0]:
            best_model[0] = acc
            best_model[1] = model
best_model   

[0.8133748055987559,
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                        max_depth=9, max_features='auto', max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=10,
                        n_jobs=None, oob_score=False, random_state=12345,
                        verbose=0, warm_start=False)]

Пробую логистическую регрессию

In [7]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
acc = accuracy_score(target_valid, predicted_valid)
if acc > best_model[0]:
    best_model[0] = acc
    best_model[1] = model
best_model

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[0.8133748055987559,
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                        max_depth=9, max_features='auto', max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=10,
                        n_jobs=None, oob_score=False, random_state=12345,
                        verbose=0, warm_start=False)]

# 4. Проверяю модель на тестовой выборке <a id="cell4"><a/>

In [8]:
predicted_test = best_model[1].predict(features_test)
print(accuracy_score(target_test, predicted_test))

0.7853810264385692


# 5. Проверяю модель на адекватность (это задание было бонусным) <a id="cell5"><a/>

не совсем понимаю, что значит адекватность, ведь точность уже была выше проверена
поступлю следующим образом:

    В марте прошлого года я проходил медицинское обследование для военкомата, и в числе прочего я прошел психиатрическое обследование, по итогу которого выяснилось, что я здоров. Таким образом, в некотором смысле, я являюсь адекватным человеком, и мои суждения, соответственно, долны быть адекватными. Для проверки адекватности моей модели я выберу выделяющиеся пользователей, у которых имеется хотя бы один показатель, лежащий ниже 1 квартиля и выше 3. Данные, принадлежащие основной массе мне рассматривать бесполезно, тк какие то адекватные суждения выносить там сложно, а статистический анализ проводить бесполезно, поскольку модель, на мой взгляд, способна учитывать статистику.

поразмыслив, решил сперва посмотреть, где же все таки расположены ошибки. оказалось -- большая их часть расположена во 2 и 3 квартиле. Значит, в том смысле, что я указал выше, модель работает адекватно. Насчет качества предсказания в целом можно спорить, обсуждать, размышлять, но поскольку требуемый порог я прошел, думаю на этом этапе можно пока остановиться.

In [9]:
mistakes = data_test[data_test['is_ultra']!=predicted_test]
calls_std, minutes_std, messages_std, mb_std = data_test['calls'].std(), data_test['minutes'].std(), data_test['messages'].std(), data_test['mb_used'].std()
#строка выше -- поиск стандартных отклонений для каждого столбца data_test
calls_median, minutes_median, messages_median, mb_median = data_test['calls'].median(), data_test['minutes'].median(), data_test['messages'].median(), data_test['mb_used'].median()
#ищу медиану
print(mistakes.shape[0], '-- число ошибок')
print('---------------------------')
print('---------------------------')
print('ошибочные строки из 2 и 3 квартиля')
mistakes[((mistakes['calls']>calls_median-calls_std) & (mistakes['calls']<calls_median+calls_std))|((mistakes['minutes']>minutes_median-minutes_std) & (mistakes['minutes']<minutes_median+minutes_std))|((mistakes['messages']>messages_median-messages_std) & (mistakes['messages']<messages_median+messages_std))|((mistakes['mb_used']>mb_median-mb_std) & (mistakes['mb_used']<mb_median+mb_std))]
     

138 -- число ошибок
---------------------------
---------------------------
ошибочные строки из 2 и 3 квартиля


,calls,minutes,messages,mb_used,is_ultra
604,10.0,96.29,20.0,3184.72,1
1094,0.0,0.00,35.0,17322.59,0
2593,98.0,761.57,0.0,21046.27,0
1231,13.0,82.55,4.0,2890.46,1
2510,63.0,508.70,21.0,19254.67,1
...,...,...,...,...,...
1628,85.0,509.61,4.0,26802.84,0
2850,46.0,309.91,105.0,6444.87,0
2066,24.0,163.09,33.0,22268.51,1
2695,92.0,614.41,49.0,16303.34,1
